# Dog cat classification

In [1]:
from nis import cat
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import torchvision.transforms as transforms
from torchvision import datasets, transforms, models
from torch.utils.data.dataloader import default_collate

In [8]:
device = torch.device("mps")

In [6]:
transform = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor()])

dataset = datasets.ImageFolder("data/train/", transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32,collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)),shuffle=True)

labels = {
    "cat",
    "dog"
}


In [9]:
images, labels = next(iter(dataloader))
images

/opt/homebrew/lib/python3.10/site-packages/torch/_tensor_str.py:103: UserWarning: The operator 'aten::bitwise_and.Tensor_out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(tensor_view, torch.isfinite(tensor_view) & tensor_view.ne(0))


tensor([[[[0.2078, 0.2078, 0.2118,  ..., 0.4549, 0.4235, 0.4235],
          [0.2039, 0.2118, 0.2157,  ..., 0.4510, 0.4275, 0.4235],
          [0.2118, 0.2118, 0.2118,  ..., 0.4588, 0.4431, 0.4039],
          ...,
          [0.6510, 0.6039, 0.6275,  ..., 0.7529, 0.6667, 0.6863],
          [0.7098, 0.7176, 0.6588,  ..., 0.7451, 0.6510, 0.6863],
          [0.7294, 0.7333, 0.7137,  ..., 0.7020, 0.6314, 0.6824]],

         [[0.1922, 0.1922, 0.1961,  ..., 0.4588, 0.4275, 0.4275],
          [0.1882, 0.1961, 0.2000,  ..., 0.4549, 0.4314, 0.4275],
          [0.1882, 0.1882, 0.1922,  ..., 0.4627, 0.4471, 0.4078],
          ...,
          [0.6000, 0.5529, 0.5765,  ..., 0.6980, 0.6118, 0.6314],
          [0.6588, 0.6667, 0.6078,  ..., 0.6902, 0.5961, 0.6314],
          [0.6784, 0.6824, 0.6667,  ..., 0.6471, 0.5765, 0.6275]],

         [[0.1804, 0.1804, 0.1843,  ..., 0.4667, 0.4353, 0.4353],
          [0.1765, 0.1843, 0.1882,  ..., 0.4627, 0.4392, 0.4353],
          [0.1843, 0.1882, 0.1882,  ..., 0

In [12]:
model_resnet = models.resnet18(pretrained = True)
model_resnet.fc = torch.nn.Sequential(
    torch.nn.Linear(2048, 1, bias = True),
)
model_resnet.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_resnet.parameters(), lr=0.001)

In [ ]:

for epoch in range(3):
    for i, data in enumerate(dataloader,0):
        inputs, labels = data
        inputs.to(device)
        labels.to(device)

        outputs = model_resnet(inputs)
        loss = loss_fn(outputs, labels)

        loss.backward()
        optimizer.step()
        print(f"Current loss {loss.item()}")